In [118]:
from transformers import ViTImageProcessor, ViTForImageClassification, ViTModel, ViTConfig
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm, trange
from torchvision import models, transforms
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split

In [92]:
images_path = os.listdir('data/images')

In [93]:
data = pd.read_csv('data/aaa_advml_final_project.csv')
data.sample(5)

,item_id,external_id,logical_category,category_name,subcategory_name,microcat_name,Param1,Param2,Param3,Param4,price,title,description,image_id
1081,2090090500977,7363938700,Goods.Fashion,Личные вещи,Часы и украшения,Ювелирные изделия,Ювелирные изделия,NaN,NaN,NaN,533325.0,Подвеска Chopard Happy Hearts 79A074-5301,⌚Подвеска Chopard Happy Hearts 79A074-5301⌚\n\...,43863699422
42160,2090012250007,7365355804,Goods.InformationTechnology,Электроника,Товары для компьютера,Видеокарты,Комплектующие,Видеокарты,NaN,NaN,6000.0,Nvidia RTX 2060 super dual palit не рабочая,Приветствую! В один момент решил заменить терм...,43861732648
11574,2089954000407,7342462323,Goods.GoodsForChildren,Личные вещи,Товары для детей и игрушки,Игрушки для малышей,Игрушки,Игрушки для малышей,NaN,NaN,450.0,Бизидомик,"В отличном состоянии, все детали на месте.\nОт...",43858490262
21108,2089929000971,7348661534,Goods.InformationTechnology,Электроника,Товары для компьютера,Сетевое оборудование,Сетевое оборудование,NaN,NaN,NaN,500.0,Wifi роутер Asus RT-N11P,а идеальном состоянии,43857496650
49907,2090085500329,7339766805,Goods.HealthAndBeauty,Личные вещи,Красота и здоровье,Бронзеры и хайлайтеры,Макияж и маникюр,Для лица,Бронзеры и хайлайтеры,NaN,1100.0,Шиммерный бронзер Terracotta,❗️В наличии\n\nОригинальный шиммерный бронзер ...,43640707845


In [94]:
data['image_path'] = 'data/images/' + data.image_id.astype(str) + '.jpg'

In [95]:
data = data[data.price >= 500]

In [96]:
data['log_price'] = np.log(data['price'])

In [131]:
class CustomHead(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    def forward(self, x):
        return self.head(x).squeeze(1)

resnet = models.resnet18(pretrained=True).to('mps')
resnet.fc = CustomHead(resnet.fc.in_features).to('mps')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [132]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
# model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

In [133]:
# class ViTForRegression(torch.nn.Module):
#     def __init__(self, pretrained_model_name='google/vit-base-patch16-224'):
#         super().__init__()
#         self.vit = ViTModel.from_pretrained(pretrained_model_name)
#         self.regressor = torch.nn.Linear(self.vit.config.hidden_size, 1)
        
#     def forward(self, pixel_values):
#         outputs = self.vit(pixel_values=pixel_values)
#         pooled_output = outputs.pooler_output
#         price = self.regressor(pooled_output)
#         return price.squeeze(-1)

In [134]:
# model = ViTForRegression().to('mps')

In [135]:
# for name, param in model.vit.named_parameters():
#     print(name)
#     param.requires_grad = False
# #     if "embeddings" not in name:  # позиционные/патч-эмбеддинги менее важны
# #         param.requires_grad = True

In [136]:
# for p in model.parameters():
#     print(p.requires_grad)

In [137]:
class PriceDataset(Dataset):
    def __init__(self, img_paths, prices, processor):
        self.img_paths = img_paths
        self.prices = prices
        self.processor = processor

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx]).convert("RGB")
        inputs = self.processor(images=image, return_tensors="pt")
        price = self.prices[idx]
        return {**{k: v.squeeze(0) for k, v in inputs.items()}, "price": torch.tensor(price, dtype=torch.float32)}

In [138]:
data_train, data_val, price_train, price_val = train_test_split(
    data[data.subcategory_name=='Одежда, обувь, аксессуары'].image_path.values,
    data[data.subcategory_name=='Одежда, обувь, аксессуары'].log_price.values, test_size=0.2)

In [139]:
dataset_train = PriceDataset(data_train, price_train, processor)
dataloader_train = DataLoader(dataset_train, batch_size=8, shuffle=True)

dataset_val = PriceDataset(data_val, price_val, processor)
dataloader_val = DataLoader(dataset_val, batch_size=8, shuffle=True)

In [141]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=1e-4)

num_epochs = 100
for epoch in range(num_epochs):
    
    resnet.train()
    running_loss = 0.0
    mae = []
    mse = []
    mape = []
    for batch in tqdm(dataloader_train):
        pixel_values = batch['pixel_values'].to('mps')
        prices = batch['price'].to('mps')

        preds = resnet(pixel_values)
        loss = loss_fn(preds, prices)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * pixel_values.size(0)
    
    resnet.eval()
    for batch in tqdm(dataloader_val):
        pixel_values = batch['pixel_values'].to('mps')
        prices = batch['price'].to('cpu').detach().numpy()

        preds = resnet(pixel_values).to('cpu').detach().numpy()
        
        mae.append(mean_absolute_error(np.exp(prices), np.exp(preds)))
        mse.append(mean_squared_error(np.exp(prices), np.exp(preds)))
        mape.append(mean_absolute_percentage_error(np.exp(prices), np.exp(preds)))
        
    avg_loss_train = running_loss / len(dataloader_train.dataset)
    print(f"Epoch {epoch+1}, Loss train: {avg_loss_train:.4f}")
    print(f"MAE val: {np.mean(mae)}, MSE val: {np.mean(mse)}, MAPE val: {np.mean(mape)}\n")


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 223/223 [00:13<00:00, 16.51it/s]


Epoch 1, Loss train: 1.0606
MAE val: 3093.5475017735776, MSE val: 166105543.7373879, MAPE val: 0.9071110875884514



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 223/223 [00:13<00:00, 16.50it/s]


Epoch 2, Loss train: 0.8543
MAE val: 3796.814748447572, MSE val: 182501537.7247758, MAPE val: 1.4941684049073891



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 223/223 [00:13<00:00, 16.68it/s]


...

Epoch 98, Loss train: 0.0313
MAE val: 3856.9271351082975, MSE val: 178717629.6191844, MAPE val: 0.7679167683883633



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 223/223 [00:13<00:00, 15.99it/s]


Epoch 99, Loss train: 0.0339
MAE val: 3862.224257670176, MSE val: 178674075.1297646, MAPE val: 0.7706055943206821



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 223/223 [00:13<00:00, 16.07it/s]

Epoch 100, Loss train: 0.0301
MAE val: 3827.522934986337, MSE val: 178245680.97954035, MAPE val: 0.7517310594763991



In [142]:
torch.save(resnet.state_dict(), 'models/resnet18_shoesСlothes_model_weights_100epoch.pth')

In [80]:
data[data.subcategory_name=='Одежда, обувь, аксессуары'].log_price

5        5.700444
7        6.214608
11       5.857933
14       5.393628
17       7.600902
           ...   
61451    8.006368
61453    6.620073
61455    6.897705
61457    7.377759
61459    6.551080
Name: log_price, Length: 10599, dtype: float64

In [106]:
item = next(iter(dataloader))['pixel_values']

In [50]:
dataset = PriceDataset(data.image_path.values[100:101], data.price.values[100:101], processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [54]:
next(iter(dataloader))['pixel_values']

tensor([[[[ 0.6235,  0.6235,  0.6078,  ..., -0.8039, -0.8039, -0.8039],
          [ 0.6157,  0.6157,  0.6000,  ..., -0.8196, -0.7804, -0.7725],
          [ 0.6078,  0.6078,  0.5922,  ..., -0.8275, -0.7882, -0.7725],
          ...,
          [ 0.6078,  0.6078,  0.6157,  ..., -0.7882, -0.7961, -0.7961],
          [ 0.6078,  0.6157,  0.6235,  ..., -0.7961, -0.7961, -0.7961],
          [ 0.6235,  0.6235,  0.6235,  ..., -0.7882, -0.7961, -0.7882]],

         [[ 0.5843,  0.5843,  0.5843,  ..., -0.8039, -0.8039, -0.8039],
          [ 0.5765,  0.5765,  0.5765,  ..., -0.8196, -0.7804, -0.7725],
          [ 0.5686,  0.5686,  0.5765,  ..., -0.8275, -0.7882, -0.7725],
          ...,
          [ 0.6000,  0.6000,  0.6078,  ..., -0.7961, -0.7961, -0.7882],
          [ 0.6000,  0.6078,  0.6157,  ..., -0.7882, -0.7882, -0.7804],
          [ 0.6157,  0.6157,  0.6157,  ..., -0.7804, -0.7804, -0.7725]],

         [[ 0.5451,  0.5373,  0.5137,  ..., -0.8196, -0.8196, -0.8196],
          [ 0.5373,  0.5294,  

In [113]:
item = next(iter(dataloader))
torch.exp(resnet(item['pixel_values'].to('mps'))), torch.exp(item['price'])

(tensor([  1056.3925,    748.4739,   3739.2834,   6787.4023,   6389.0664,
           4000.2693,  19018.6270,   4385.4790,   1310.2233,  12394.5439,
           7493.3701, 141591.5156,   1299.7678,  11087.5332,   1387.0173,
           4886.9180], device='mps:0', grad_fn=<ExpBackward0>),
 tensor([  1000.0001,    600.0001,    550.0001,   6499.9985,  35000.0000,
           3872.9995,  27999.9980,   4999.9980,    513.0000,  27499.9922,
           3259.9995, 101727.0312,    500.0001,  14499.9932,    500.0001,
           3300.0005]))

In [114]:
torch.save(resnet.state_dict(), 'models/resnet18_shoesСlothes_model_weights.pth')

In [116]:
from sklearn.metrics import (mean_absolute_percentage_error, mean_absolute_error, 
                             mean_squared_error, median_absolute_error)

resnet.eval()
table_metrics = pd.DataFrame({'Batch number': [], 
                              'MSE': [],
                              'MAE': [],
                              'MAPE': []})

iteration_n = [1]
preds = []
mae = []
mse = []
mape = []
for batch in tqdm(dataloader):
    pixel_values = batch['pixel_values'].to('mps')
    prices = batch['price'].to('cpu').detach().numpy()

    preds = resnet(pixel_values).to('cpu').detach().numpy()
    mae.append(mean_absolute_error(np.exp(prices), np.exp(preds)))
    mse.append(mean_squared_error(np.exp(prices), np.exp(preds)))
    mape.append(mean_absolute_percentage_error(np.exp(prices), np.exp(preds)))
    if iteration_n[-1] + 1 == 15:
        break
    iteration_n.append(iteration_n[-1] + 1)
        
table_metrics['Batch number'] = iteration_n
table_metrics['MSE'] = mse
table_metrics['MAE'] = mae
table_metrics['MAPE'] = mape
table_metrics

  2%|██▋                                                                                                               | 13/556 [00:02<01:36,  5.65it/s]


,Batch number,MSE,MAE,MAPE
0,1,4.303330e+07,3565.124023,1.071447
1,2,1.469356e+11,105609.031250,1.252380
2,3,2.494833e+08,6022.226074,0.671070
3,4,7.976246e+07,3990.212891,0.361198
4,5,1.137767e+12,280259.187500,1.478979
5,6,7.439646e+07,4419.518555,0.520931
6,7,3.437515e+09,20792.128906,0.816908
7,8,3.480012e+07,2829.015381,0.558880
8,9,6.620944e+09,34280.742188,1.180114
9,10,5.289091e+08,6388.354492,0.307588
